# Capstone Assignment Week 3

This notebook is built to prepare location information of Toronto for later use

### Part 1. Scraping the postal codes from wikipedia

Source: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Library using: beautifulsoup

Installing beautifulsoup:

In [5]:
! conda install beautifulsoup4
! conda install lxml

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.5.15  |                0         133 KB
    beautifulsoup4-4.6.3       |           py35_0         140 KB
    ------------------------------------------------------------
                                           Total:         273 KB

The following packages will be UPDATED:

    beautifulsoup4:  4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0
    ca-certificates: 2019.1.23-0          --> 2019.5.15-0 


ca-certificates-2019 | 133 KB    | ##################################### | 100% 
beautifulsoup4-4.6.3 | 140 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package 

__Load libraries and scrape the whole page, then pick out the table.__

In [31]:
# Loading libraries
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
# Getting the webpage html code
wiki = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
# Loading html
soup = BeautifulSoup(wiki, 'lxml')
print(soup.title) # Check if source codes are successfully loaded
# Pull out the table
table = soup.find('table', class_='wikitable sortable')


<title>List of postal codes of Canada: M - Wikipedia</title>


**Pull out table headers and contents and store the table into a pandas data frame.**

In [32]:
# Pull out table header
column=[]
for x in table.find_all('th'):
    column.append(x.get_text())

print(column) # Check column names

# Find out total rows including the headers
row_count=0
for x in table.find_all('tr'):
    row_count+=1
print(row_count)
row_count-=1 # Adjust for table content rows

# Set up dataframe
postal = pd.DataFrame(columns=column, index=range(0, row_count))

# Fill in table contents
row_marker = 0
header = True
for row in table.find_all('tr'):
    if header: # Skipping header
        header = False
    else:
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            postal.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        row_marker += 1

# Fixing '\n':
postal.rename(columns = {'Neighbourhood\n':'Neighborhood'}, inplace = True)  
postal['Neighborhood'] = postal['Neighborhood'].str.replace(r'\n', '') 
print(postal.shape)
postal.tail()

['Postcode', 'Borough', 'Neighbourhood\n']
289
(288, 3)


,Postcode,Borough,Neighborhood
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor
287,M9Z,Not assigned,Not assigned


__Cleaning the table to generate unique postcode lines with assigned borough names__

In [34]:
# Removing lines with boroughs "Not assigned".
print("Borough not assigned rows: ", (postal.Borough == 'Not assigned').sum()) # Check number of rows in question
postal.drop(postal.index[postal['Borough'] == 'Not assigned'], inplace = True)
print(postal.shape) # Result check

# Assigning borough name as neighbourhood name if neighbourhood name is not assigned
print("Neighborhood not assigned rows: ", (postal.Neighborhood == 'Not assigned').sum()) # Check number of rows without assigned neighbourhood
postal.loc[postal['Neighborhood'] == 'Not assigned', 'Neighborhood'] = postal.loc[postal['Neighborhood'] == 'Not assigned', 'Borough']
print("Neighborhood not assigned rows: ", (postal.Neighborhood == 'Not assigned').sum())


Borough not assigned rows:  0
(211, 3)
Neighborhood not assigned rows:  1
Neighborhood not assigned rows:  0


Reframe the data to combine neighborhoods with the same postal code

In [43]:
# Stack on postal codes and combining neighbourhoods
coderow = len(postal.Postcode.unique()) # Check number of unique postal codes
print(coderow)
stack = pd.DataFrame(columns=list(postal.columns), index=range(0, coderow))

code_array = postal.Postcode.unique() # Unique postal codes
row_marker = 0
# Fill the new frame with corresponding boroughs and joined unique neighborhoods of each postal codes.
for code in code_array:
    stack.iat[row_marker, 0] = code
    stack.iat[row_marker, 1] = ', '.join(postal.loc[postal['Postcode'] == code, 'Borough'].unique())
    stack.iat[row_marker, 2] = ', '.join(postal.loc[postal['Postcode'] == code, 'Neighborhood'].unique())
    row_marker += 1

# Check results
print(stack.head(10))
print(stack.tail(10))
print(stack.shape)

103
  Postcode           Borough                      Neighborhood
0      M3A        North York                         Parkwoods
1      M4A        North York                  Victoria Village
2      M5A  Downtown Toronto         Harbourfront, Regent Park
3      M6A        North York  Lawrence Heights, Lawrence Manor
4      M7A      Queen's Park                      Queen's Park
5      M9A         Etobicoke                  Islington Avenue
6      M1B       Scarborough                    Rouge, Malvern
7      M3B        North York                   Don Mills North
8      M4B         East York   Woodbine Gardens, Parkview Hill
9      M5B  Downtown Toronto          Ryerson, Garden District
    Postcode           Borough  \
93       M8W         Etobicoke   
94       M9W         Etobicoke   
95       M1X       Scarborough   
96       M4X  Downtown Toronto   
97       M5X  Downtown Toronto   
98       M8X         Etobicoke   
99       M4Y  Downtown Toronto   
100      M7Y      East Toronto 

__The target table is generated with 103 rows of postal codes with corresponding boroughs and neighborhoods.__

__End of this part__